In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

dde.config.set_random_seed(1234)

In [ ]:
num_dense_layers = 5
num_dense_nodes = 20
iterations = 10000
activation = 'tanh'
initializer = 'Glorot uniform'
weights = 100

In [ ]:
num_train = 32
num_test = 100
num_boundary = 2

In [ ]:
L = 2
n = 5
k = (n * np.pi) / L
E = 0.5 * (k ** 2)

In [ ]:
domain = dde.geometry.Interval(-L / 2, L / 2)

In [ ]:
def pde(x, y):
    dy_xx = dde.grad.hessian(y, x)
    return 0.5 * dy_xx + E * y

In [ ]:
def psi(x):
    normalization_constant = np.sqrt(2.0 / L)
    return normalization_constant * np.sin(k * (x + 0.5 * L))

In [ ]:
def get_collocation_points(n):
    points = []
    for k in range(n):
        x = (k * L) / n + L / (2 * n) - L / 2
        points.append([x])
        
    if n == 1:
        points.append([L / 4])
        points.append([-L / 4])
    
    return np.array(points)

In [ ]:
collocation_points = get_collocation_points(n)
collocation_values = psi(collocation_points)
ic = dde.icbc.PointSetBC(collocation_points, collocation_values)

In [ ]:
def x_boundary(_, on_boundary):
    return on_boundary

dirichlet_bc = dde.icbc.DirichletBC(domain, lambda x: 0, x_boundary)

In [ ]:
data = dde.data.PDE(
    domain, 
    pde, 
    [ic, dirichlet_bc], 
    num_domain=num_train, 
    num_boundary=num_boundary,
    solution=psi, 
    num_test=num_test
)

In [ ]:
net = dde.nn.FNN(
    [1] + [num_dense_nodes] * num_dense_layers + [1], 
    activation, 
    initializer
)

In [ ]:
model = dde.Model(data, net)

loss_weights = [1, weights, weights]
model.compile(
    'L-BFGS', 
    metrics=['l2 relative error'],
    loss_weights=loss_weights
)

In [ ]:
loss_history, train_state = model.train(iterations=iterations)

In [ ]:
dde.saveplot(loss_history, train_state, issave=True, isplot=True)

In [ ]:
def save_loss_plot():
    loss_file = open('loss.dat', 'r')
    
    loss_file.readline()
    steps = []
    train_loss = []
    test_loss = []
    test_metric = []
    
    for line in loss_file:
        line = line.split()
        steps.append(float(line[0]))
        train_loss.append(max([float(line[i]) for i in range(1, 4)]))
        test_loss.append(max([float(line[i]) for i in range(4, 7)]))
        test_metric.append(float(line[-1]))
    
    loss_file.close()
    
    plt.plot(steps, train_loss, label='Train loss')
    plt.plot(steps, test_loss, label='Test loss')
    plt.plot(steps, test_metric, label='Test metric')
    
    plt.xlabel('training epoch')
    plt.yscale('log')
    
    plt.legend()
    plt.savefig('{}-loss.png'.format(n))
    plt.show()

In [ ]:
def save_prediction_plot():
    train_file = open('train.dat', 'r')
    
    x_train = []
    y_train = []
    train_file.readline()
    
    for line in train_file:
        line = line.split()
        x_train.append(float(line[0]))
        y_train.append(float(line[1]))
    
    train_file.close()
    
    test_file = open('test.dat', 'r')
    
    x_test = []
    y_true = []
    y_pred = []
    test_file.readline()
    
    for line in test_file:
        line = line.split()
        x_test.append(float(line[0]))
        y_true.append(float(line[1]))
        y_pred.append(float(line[2]))
    
    test_file.close()
    
    x_true, y_true = zip(*sorted(zip(x_test, y_true)))
    x_pred, y_pred = zip(*sorted(zip(x_test, y_pred)))
    
    plt.plot(x_train, y_train, 'o', color='black', label='Training points')
    plt.plot(x_true, y_true, '-', color='black', label='True values')
    plt.plot(x_pred, y_pred, '--', color='red', label='Predicted values')
    
    plt.xlabel('x')
    plt.ylabel('$\psi_{}$(x)'.format(n))
    
    plt.legend()
    plt.savefig('{}-results.png'.format(n))
    plt.show()

In [ ]:
save_prediction_plot()

In [ ]:
save_loss_plot()

In [ ]:
!mv loss.dat $n-loss.dat
!mv test.dat $n-test.dat
!mv train.dat $n-train.dat